In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import json
from loaders import *

In [ ]:
res_df, scalars = adaptive_ratio_v1_0_loader()
res_wm_df, scalars_wm = adaptive_ratio_wm_v1_0_loader()
res_b_df, scalars_b = baseline_loader()

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    rows=1,
    cols=len(envs),
    column_titles=[*envs],
)


colors = make_color_iter()
colors = [next(colors) for _ in range(2)]
axis = 1

for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    df1["tag"] = "Ada"
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            name="Ada",
            offsetgroup="0",
            legendgroup="0",
            showlegend=(col == 1),
            marker=dict(color=colors[0]),
        ),
        row=1,
        col=col,
    )

    df2 = res_b_df[res_b_df["env"] == env].copy()
    df2["tag"] = df2["ratio"]
    df2 = df2.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["score"],
            sizemode="sd",
            boxmean=True,
            name="Base",
            offsetgroup="1",
            legendgroup="1",
            showlegend=(col == 1),
            marker=dict(color=colors[1]),
        ),
        row=1,
        col=col,
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Configuration")})
    axis += 1

fig.update_layout(width=1000, height=400, yaxis_title="Score")
fig.write_image("../tex/assets/adaptive_v1_0.pdf")
fig

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    rows=1,
    cols=len(envs),
    column_titles=[*envs],
)


def add_val_losses(df, scalars, key="val/wm_loss"):
    values = []
    for _, row in df.iterrows():
        test_df = scalars.read(row["path"])
        val_losses = test_df[test_df["tag"] == key]
        val_losses = val_losses["value"].to_numpy()
        values.append(val_losses[-1])
    df["val_loss"] = values


val_ranges = {
    "Assault": [1.5, 5.0],
    "CrazyClimber": [0.5, 1.5],
    "MsPacman": [1.2, 2.5],
}


colors = make_color_iter()
colors = [next(colors) for _ in range(2)]
axis = 1
for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    add_val_losses(df1, scalars)
    df1["tag"] = "Ada"
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            name="Ada",
            offsetgroup="0",
            legendgroup="0",
            showlegend=(col == 1),
            marker=dict(color=colors[0]),
        ),
        row=1,
        col=col,
    )

    df2 = res_b_df[res_b_df["env"] == env].copy()
    add_val_losses(df2, scalars_b)
    df2["tag"] = df2["ratio"]
    df2 = df2.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["score"],
            sizemode="sd",
            boxmean=True,
            name="Base",
            offsetgroup="1",
            legendgroup="1",
            showlegend=(col == 1),
            marker=dict(color=colors[1]),
        ),
        row=1,
        col=col,
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Configuration")})
    axis += 1


fig.update_layout(width=1000, height=400, yaxis_title="Score")
fig.write_image("../tex/assets/adaptive_v1_0.val_loss.pdf")
fig

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    rows=2,
    cols=len(envs),
    column_titles=[*envs],
)

val_ranges = {
    "Assault": [1.5, 5.0],
    "CrazyClimber": [0.5, 1.5],
    "MsPacman": [1.4, 2.5],
}


color_iter = make_color_iter()
colors = [next(color_iter) for _ in range(5)]


# Loss curves
for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    dfs = []
    for _, row in df1.iterrows():
        test_df = scalars.read(row["path"])
        test_df = test_df[test_df["tag"] == "ada/val_loss"].copy()
        test_df["seed"] = row["seed"]
        dfs.append(test_df)
    df = pd.concat(dfs)

    for seed in range(2):
        seed_df = df[df["seed"] == seed]
        fig.add_trace(
            go.Scatter(
                x=seed_df["step"],
                y=seed_df["value"],
                legendgroup=f"Seed = {seed}",
                name=f"Seed = {seed}",
                line=dict(color=colors[seed]),
                showlegend=(col == 1),
            ),
            row=1,
            col=col,
        )

    axis = col
    yaxis_name = f"yaxis{axis if axis > 1 else ''}"
    fig.update_layout(**{yaxis_name: dict(range=val_ranges[env])})


# Loss curves
for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    dfs = []
    for _, row in df1.iterrows():
        if row["seed"] >= 2:
            continue
        test_df = scalars.read(row["path"])
        test_df = test_df[test_df["tag"] == "ada/wm_ratio"].copy()
        test_df["seed"] = row["seed"]
        dfs.append(test_df)
    df = pd.concat(dfs)

    for seed in range(2):
        seed_df = df[df["seed"] == seed]
        fig.add_trace(
            go.Scatter(
                x=seed_df["step"],
                y=seed_df["value"],
                legendgroup=f"Seed = {seed}",
                name=f"Seed = {seed}",
                line=dict(color=colors[seed]),
                showlegend=False,
            ),
            row=2,
            col=col,
        )

    fig.update_layout(**{f"yaxis{3 + col}": dict(type="log")})

fig.update_layout(
    width=1000,
    height=500,
    yaxis1=dict(title="Val loss"),
    yaxis4=dict(title="Update ratio"),
    **{f"xaxis{a}": dict(title="Env step") for a in range(4, 7)},
)

fig.update_layout(width=1000, height=500)
fig.write_image("../tex/assets/adaptive_v1_0.curves.pdf")
fig

In [ ]:
envs = res_df["env"].unique()

fig = make_subplots(
    rows=1,
    cols=len(envs),
    column_titles=[*envs],
)


colors = make_color_iter()
colors = [next(colors) for _ in range(2)]
axis = 1

for col, env in enumerate(envs, 1):
    df1 = res_df[res_df["env"] == env].copy()
    tag = "WM and RL"
    df1["tag"] = tag
    df1 = df1.sort_values(by="tag")

    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            name="Coupled",
            offsetgroup="0",
            legendgroup="0",
            showlegend=(col == 1),
            marker=dict(color=colors[0]),
        ),
        row=1,
        col=col,
    )

    df2 = res_wm_df[res_wm_df["env"] == env].copy()
    tag = df2["rl_ratio"].apply(lambda x: f"WM only (K = {x})")
    df2["tag"] = tag
    df2 = df2.sort_values(by="tag")

    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["score"],
            sizemode="sd",
            boxmean=True,
            name="Decoupled",
            offsetgroup="1",
            legendgroup="1",
            showlegend=(col == 1),
            marker=dict(color=colors[1]),
        ),
        row=1,
        col=col,
    )

fig.update_layout(width=1000, height=400, yaxis_title="Score")
fig.write_image("../tex/assets/adaptive_wm_v1_0.pdf")
fig